In [1]:
import pandas as pd
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
x = np.load("./x_(7727,10,4069).npy")
x_2d = np.load('./x_(7727,4069).npy')
y = np.load("./y_(7727,1).npy")
x.shape, x.sum()
x_2d.shape, x_2d.sum()
y.shape, y.sum()

((7727, 10, 4069), 2042722.0)

((7727, 4069), 510420.0)

((7727,), 4718)

In [4]:
seed_num = 43

import random
random.seed(seed_num)

# 80% (6239) 기준으로 split

idx = list(range(len(x)))
random.shuffle(idx)

i= round(len(x)*0.8)

X_train = x[idx[:i],:,:]
y_train = np.array(y[idx[:i]])

X_test = x[idx[i:],:,:]
y_test = np.array(y[idx[i:]])

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [11]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

tf.random.set_seed(seed_num)
lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])

from tqdm import tqdm
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier

#LSTM 쌓기
estimator = []
for i in tqdm(range(1,6)): #일단 1개로 정확도 측정?? 
    LSTM_Predictors = KerasClassifier(build_fn=lambda:lstm, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

100%|██████████| 5/5 [00:00<00:00, 10580.99it/s]


[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bf70>), ('model2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bf40>), ('model3', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bfa0>), ('model4', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bdf0>), ('model5', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812be20>)]
Epoch 1/20
25/25 [==============================] - 18s 99ms/step - loss: 0.7310 - acc: 0.4837
Epoch 2/20
25/25 [==============================] - 2s 97ms/step - loss: 0.6792 - acc: 0.5957
Epoch 3/20
25/25 [==============================] - 2s 97ms/step - loss: 0.6699 - acc: 0.6153
Epoch 4/20
25/25 [==============================] - 2s 95ms/step - loss: 0.6636 - acc: 0.6249
Epoch 5/20
25/25 [==============================] - 2s 96ms/step - loss: 0.6432 - acc: 0.6253
Epoch 6/20
25/25 [==============================] - 2s 98ms/step - loss: 0.5733 - acc: 0.7051
Epoch 7/20
25/

VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bf70>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bf40>),
                             ('model3',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bfa0>),
                             ('model4',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812bdf0>),
                             ('model5',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8cf812be20>)],
                 voting='soft')

In [12]:
#예측 후 가공
preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
# print("RMSE : ", RMSE(y_test, preds))

정확도 : 0.7398058252427184
